In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from functions import *
from jacobi import *

In [10]:
%load_ext autoreload
%autoreload 2

In [103]:
%%bash
rm -rf ../build/
mkdir -p ../build
cd ../build
cmake ../ -DCMAKE_BUILD_TYPE=Release
cmake --build . -j4
cd -

-- The C compiler identification is GNU 13.3.0
-- The CXX compiler identification is GNU 13.3.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.83.0/BoostConfig.cmake (found version "1.83.0") found components: program_options 
-- HDF5 CXX compiler wrapper is unable to compile a minimal HDF5 program.
-- Found HDF5: /usr/lib/x86_64-linux-gnu/hdf5/serial/libhdf5_cpp.so;/usr/lib/x86_64-linux-gnu/hdf5/serial/libhdf5.so (found version "1.10.10") found components: CXX 
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found Ope

In [104]:
%%bash
cd ../build/
ctest --output-on-failure
cd -

Test project /home/balazs/Desktop/University/Research/Practice/Laplace/build
      Start  1: HDF5WriterTest.WritesAndReadsHDF5
 1/19 Test  #1: HDF5WriterTest.WritesAndReadsHDF5 .............   Passed    0.01 sec
      Start  2: CalculatorTest.ConvergesOnKnownGrid
 2/19 Test  #2: CalculatorTest.ConvergesOnKnownGrid ...........   Passed    0.01 sec
      Start  3: FileHandlerTest.ThrowsOnNonExistentFile
 3/19 Test  #3: FileHandlerTest.ThrowsOnNonExistentFile .......   Passed    0.00 sec
      Start  4: FileHandlerTest.ReadsAndWritesCSVCorrectly
 4/19 Test  #4: FileHandlerTest.ReadsAndWritesCSVCorrectly ....   Passed    0.00 sec
      Start  5: HDF5WriterTest.WritesParametersCorrectly
 5/19 Test  #5: HDF5WriterTest.WritesParametersCorrectly ......   Passed    0.01 sec
      Start  6: HDF5WriterTest.WritesResultDatasetCorrectly
 6/19 Test  #6: HDF5WriterTest.WritesResultDatasetCorrectly ...   Passed    0.01 sec
      Start  7: HDF5WriterTest.CreatesExpectedFileStructure
 7/19 Test  #7: HDF

In [105]:
generate_ex1(10, 15, "Data")

In [119]:
!time ../build/src/laplace_serial -h


Generic options:
  -h [ --help ]               Show the help message
  --config arg                Path to the config file

System options:
  --N arg                     Width of the grid
  --M arg                     Height of the grid
  --initial-state arg         Path to the initial state file
  --mask arg                  Path to the mask file
  --result-path arg           Path to save the results file
  --tol arg (=0.00100000005)  Tolerance
  --max-iter arg (=1000)      Maximum number of iterations
  --output arg (=2)           Amount of information printed


real	0m0,007s
user	0m0,002s
sys	0m0,005s


In [108]:
!time ../build/src/laplace_serial --config ../examples/config.txt --result-path ../examples/Data/N10_M15_res_serial

OpenMP NOT enabled.
Results saved to file: ../examples/Data/N10_M15_res_serial, using 55 iterations.
bye

real	0m0,012s
user	0m0,008s
sys	0m0,009s


In [110]:
%%bash
export OMP_NUM_THREADS=3
time ../build/src/laplace_serial --config ../examples/config.txt --result-path ../examples/Data/N10_M15_res_openMP_3

OpenMP NOT enabled.
Results saved to file: ../examples/Data/N10_M15_res_openMP_3, using 55 iterations.
bye



real	0m0,010s
user	0m0,006s
sys	0m0,004s


In [111]:
!time python jacobi.py --config config.txt --result-path ../examples/Data/N10_M15_res_python.csv

Results saved to file: ../examples/Data/N10_M15_res_python.csv.
bye

real	0m0,145s
user	0m1,122s
sys	0m0,058s


In [112]:
!clang-format -i ../src/*.cpp ../src/*.hpp ../test/*.cpp

In [118]:
file_list = ["Data/N10_M15_res_python.csv", 
             "Data/N10_M15_res_serial.csv", "Data/N10_M15_res_serial.h5",
             #"Data/N10_M15_res_openMP_1.csv", "Data/N10_M15_res_openMP_1.h5",
             #"Data/N10_M15_res_openMP_2.csv", "Data/N10_M15_res_openMP_2.h5",
             "Data/N10_M15_res_openMP_3.csv", "Data/N10_M15_res_openMP_3.h5",
             #"Data/N10_M15_res_openMP_4.csv", "Data/N10_M15_res_openMP_4.h5",
             ]
reference_file = "Data/N10_M15_res_python.csv"
compare_arrays(file_list, reference_file, rtol=1e-5, atol=1e-8)

[PASS] Data/N10_M15_res_serial.csv is close to Data/N10_M15_res_python.csv
[PASS] Data/N10_M15_res_serial.h5 is close to Data/N10_M15_res_python.csv
[PASS] Data/N10_M15_res_openMP_3.csv is close to Data/N10_M15_res_python.csv
[PASS] Data/N10_M15_res_openMP_3.h5 is close to Data/N10_M15_res_python.csv


True